In [5]:
from easy21 import *
import numpy as np

In [8]:
class MC_Agent:
    def __init__(self, environment, n0):
        self.n0 = float(n0)
        self.env = environment
        
        # initialize tables for (state, action) pairs occurrences, values, eligibility
        self.N = np.zeros((self.env.dealer_values_count,
                           self.env.player_values_count, 
                           self.env.actions_count))
        self.Q = np.zeros((self.env.dealer_values_count,
                           self.env.player_values_count, 
                           self.env.actions_count))
        # self.E = np.zeros((self.env.dealer_values_count,
        #                    self.env.player_values_count, 
        #                    self.env.actions_count))

         # init values
        self.V = np.zeros((self.env.dealer_values_count, self.env.player_values_count))
        
        self.count_wins = 0

    def get_action(self, s):
        a = Actions.hit
        return a

    def train(self, iterations):        
        
        # Loop episodes
        for episode in xrange(iterations):
            episode_pairs = []
            
            # get initial state for current episode
            s = self.env.get_start_state()
            
            # Execute until game ends
            while not s.term:
                
                # get action with epsilon greedy policy
                a = self.get_action(s)
                
                # store action state pairs
                episode_pairs.append((s, a))
                
                # update visits
                self.N[s.dealer-1, s.player-1, Actions.as_int(a)] += 1
                
                # execute action
                s,r = self.env.step(s, a)

            #if episode%10000==0: print "Episode: %d, Reward: %d" %(episode, my_state.rew)
            self.count_wins = self.count_wins+1 if r==1 else self.count_wins

            # Update Action value function accordingly
            for s, a in episode_pairs:
                print s.dealer, s.player, s.r, a
                step = 1.0 / (self.N[s.dealer-1, s.player-1, Actions.as_int(a)])
                error = s.r - self.Q[s.dealer-1, s.player-1, Actions.as_int(a)]
                self.Q[s.dealer-1, s.player-1, Actions.as_int(a)] += step * error

        print float(self.count_wins)/iterations*100

        # Derive value function
        for d in xrange(self.env.dealer_values_count):
            for p in xrange(self.env.player_values_count):
                self.V[d,p] = max(self.Q[d, p, :])
                


# TODO 
#  add missing values
#  make train so it can be ran multiple sets of itteractions
#   (and figure out when to compute the value function and to make sure its done from start of file

In [9]:
agent = MC_Agent(Environment(),100)
agent.train(1000)

2 10 0 Actions.hit
2 6 0 Actions.hit
2 5 0 Actions.hit
2 3 0 Actions.hit
2 4 0 Actions.hit
2 9 0 Actions.hit
2 10 0 Actions.hit
2 6 0 Actions.hit
2 15 0 Actions.hit
2 19 0 Actions.hit
2 14 0 Actions.hit
2 8 0 Actions.hit
2 11 0 Actions.hit
2 3 0 Actions.hit
2 13 0 Actions.hit
2 11 0 Actions.hit
2 20 0 Actions.hit
5 7 0 Actions.hit
5 15 0 Actions.hit
6 8 0 Actions.hit
6 7 0 Actions.hit
6 13 0 Actions.hit
6 14 0 Actions.hit
6 20 0 Actions.hit
3 8 0 Actions.hit
3 12 0 Actions.hit
3 19 0 Actions.hit
3 6 0 Actions.hit
3 7 0 Actions.hit
3 15 0 Actions.hit
3 13 0 Actions.hit
3 6 0 Actions.hit
3 16 0 Actions.hit
3 17 0 Actions.hit
6 5 0 Actions.hit
6 3 0 Actions.hit
6 8 0 Actions.hit
3 7 0 Actions.hit
3 6 0 Actions.hit
4 3 0 Actions.hit
9 5 0 Actions.hit
9 14 0 Actions.hit
6 4 0 Actions.hit
6 14 0 Actions.hit
10 10 0 Actions.hit
10 19 0 Actions.hit
10 16 0 Actions.hit
10 9 0 Actions.hit
8 3 0 Actions.hit
8 13 0 Actions.hit
8 2 0 Actions.hit
8 9 0 Actions.hit
8 3 0 Actions.hit
8 5 0 Actions.hit

In [7]:
s = State(Card(), Card())
print s

<easy21.State instance at 0x104b47290>
